# Links to Dataset

###### Kaggle Dataset - https://www.kaggle.com/datasets/sanjanchaudhari/employees-performance-for-hr-analytics

In [1]:
import pypyodbc as odbc

In [2]:
DRIVER_NAME = "SQL Server"
SERVER_NAME = "LAPTOP-CT9C24F3"
DATABASE_NAME = "HR Analytics"

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    Trust_Connection=yes;
"""

In [3]:
conn = odbc.connect(connection_string)
cursor = conn.cursor()

In [20]:
cursor.execute("SELECT * FROM Employees")

In [21]:
n = 0
for i in cursor:
    print(i)
    n += 1
    if n == 5:
        break

(78283.0, 'Operations', 'region_12', 'Bachelors', 'f', 'sourcing', 1.0, 28.0, 3.0, 4.0, 0.0, 0.0, 57.0)
(7656.0, 'Sales & Marketing', 'region_10', 'Masters & above', 'm', 'other', 1.0, 46.0, 4.0, 4.0, 1.0, 0.0, 59.0)
(5143.0, 'Operations', 'region_29', 'Bachelors', 'm', 'sourcing', 1.0, 35.0, 4.0, 6.0, 1.0, 0.0, 64.0)
(2981.0, 'Operations', 'region_9', 'Bachelors', 'f', 'sourcing', 1.0, 36.0, 5.0, 8.0, 1.0, 0.0, 59.0)
(40452.0, 'Sales & Marketing', 'region_22', 'Masters & above', 'm', 'sourcing', 1.0, 53.0, 3.0, 17.0, 0.0, 0.0, 47.0)
